载入特征文件（）并建立网络，分类，进行训练，

In [1]:
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(0)

In [2]:
train_features = []
test_features = []
train_labels = []

with h5py.File("xception_bottleneck.h5", 'r') as h:
    train_features.append(np.array(h['train_features']))
    test_features.append(np.array(h['test_features']))
    train_labels = np.array(h['train_label'])

In [3]:
train_features = train_features[0]
test_features = test_features[0]

In [4]:
#验证数据集的形状是否正确
print("train_features shape:",np.shape(train_features))
print("test_features shape:",np.shape(test_features))
print("train_labels shape:",np.shape(train_labels))

train_features shape: (20, 2048)
test_features shape: (10, 2048)
train_labels shape: (20,)


In [5]:
#是否打乱
train_features, train_labels = shuffle(train_features, train_labels)

构建网络

In [6]:
from keras.layers import Input, Dropout, Dense
from keras.models import Model
from keras import optimizers
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [7]:
inputlayer = Input(shape=(train_features.shape[1:]))

In [8]:
x = inputlayer

x = Dropout(0.5)(x)

#x = Dense(512, activation='relu')(x)
#x = Dropout(0.85)(x)
outlayer = Dense(1, activation='sigmoid')(x)

model = Model(input=inputlayer, output=outlayer)

/Users/liujiaye/anaconda3/envs/tensorflow2/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  if __name__ == '__main__':


In [9]:
#优化器参考：https://keras.io/optimizers/
opt = optimizers.SGD(lr=0.01, momentum=0.9)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(train_features,train_labels,nb_epoch=200,verbose=1,validation_split=0.2, batch_size=128, callbacks=[TensorBoard(log_dir='./log')])
model.save('model.h5')

/Users/liujiaye/anaconda3/envs/tensorflow2/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  after removing the cwd from sys.path.


Train on 16 samples, validate on 4 samples
Epoch 1/200
16/16 [==============================] - 0s 2ms/step - loss: 0.6912 - acc: 0.5625 - val_loss: 0.7039 - val_acc: 0.5000
Epoch 2/200
16/16 [==============================] - 0s 246us/step - loss: 0.6912 - acc: 0.5625 - val_loss: 0.7037 - val_acc: 0.5000
Epoch 3/200
16/16 [==============================] - 0s 198us/step - loss: 0.6902 - acc: 0.4375 - val_loss: 0.7028 - val_acc: 0.5000
Epoch 4/200
16/16 [==============================] - 0s 208us/step - loss: 0.7173 - acc: 0.4375 - val_loss: 0.7016 - val_acc: 0.2500
Epoch 5/200
16/16 [==============================] - 0s 249us/step - loss: 0.6810 - acc: 0.6250 - val_loss: 0.7014 - val_acc: 0.5000
Epoch 6/200
16/16 [==============================] - 0s 207us/step - loss: 0.7083 - acc: 0.5625 - val_loss: 0.7024 - val_acc: 0.5000
Epoch 7/200
16/16 [==============================] - 0s 228us/step - loss: 0.6891 - acc: 0.5000 - val_loss: 0.7027 - val_acc: 0.5000
Epoch 8/200
16/16 [=========

Epoch 62/200
16/16 [==============================] - 0s 258us/step - loss: 0.6951 - acc: 0.5000 - val_loss: 0.6907 - val_acc: 0.5000
Epoch 63/200
16/16 [==============================] - 0s 317us/step - loss: 0.6458 - acc: 0.6875 - val_loss: 0.6902 - val_acc: 0.7500
Epoch 64/200
16/16 [==============================] - 0s 423us/step - loss: 0.7113 - acc: 0.5000 - val_loss: 0.6898 - val_acc: 0.7500
Epoch 65/200
16/16 [==============================] - 0s 324us/step - loss: 0.7019 - acc: 0.3750 - val_loss: 0.6893 - val_acc: 0.7500
Epoch 66/200
16/16 [==============================] - 0s 525us/step - loss: 0.6973 - acc: 0.5625 - val_loss: 0.6889 - val_acc: 0.7500
Epoch 67/200
16/16 [==============================] - 0s 443us/step - loss: 0.7780 - acc: 0.5000 - val_loss: 0.6891 - val_acc: 0.5000
Epoch 68/200
16/16 [==============================] - 0s 288us/step - loss: 0.7916 - acc: 0.3125 - val_loss: 0.6894 - val_acc: 0.5000
Epoch 69/200
16/16 [==============================] - 0s 489us

Epoch 123/200
16/16 [==============================] - 0s 301us/step - loss: 0.6296 - acc: 0.6875 - val_loss: 0.7041 - val_acc: 0.5000
Epoch 124/200
16/16 [==============================] - 0s 468us/step - loss: 0.7446 - acc: 0.5000 - val_loss: 0.7028 - val_acc: 0.5000
Epoch 125/200
16/16 [==============================] - 0s 308us/step - loss: 0.6426 - acc: 0.6250 - val_loss: 0.7028 - val_acc: 0.5000
Epoch 126/200
16/16 [==============================] - 0s 409us/step - loss: 0.7541 - acc: 0.4375 - val_loss: 0.7028 - val_acc: 0.7500
Epoch 127/200
16/16 [==============================] - 0s 466us/step - loss: 0.6839 - acc: 0.6250 - val_loss: 0.7037 - val_acc: 0.2500
Epoch 128/200
16/16 [==============================] - 0s 330us/step - loss: 0.7676 - acc: 0.5000 - val_loss: 0.7047 - val_acc: 0.2500
Epoch 129/200
16/16 [==============================] - 0s 280us/step - loss: 0.7894 - acc: 0.3750 - val_loss: 0.7060 - val_acc: 0.2500
Epoch 130/200
16/16 [==============================] - 

Epoch 184/200
16/16 [==============================] - 0s 283us/step - loss: 0.7054 - acc: 0.5625 - val_loss: 0.7117 - val_acc: 0.2500
Epoch 185/200
16/16 [==============================] - 0s 533us/step - loss: 0.6211 - acc: 0.6875 - val_loss: 0.7112 - val_acc: 0.2500
Epoch 186/200
16/16 [==============================] - 0s 764us/step - loss: 0.7400 - acc: 0.3750 - val_loss: 0.7121 - val_acc: 0.2500
Epoch 187/200
16/16 [==============================] - 0s 505us/step - loss: 0.6715 - acc: 0.6250 - val_loss: 0.7134 - val_acc: 0.5000
Epoch 188/200
16/16 [==============================] - 0s 358us/step - loss: 0.6526 - acc: 0.5625 - val_loss: 0.7151 - val_acc: 0.5000
Epoch 189/200
16/16 [==============================] - 0s 326us/step - loss: 0.7295 - acc: 0.3750 - val_loss: 0.7166 - val_acc: 0.5000
Epoch 190/200
16/16 [==============================] - 0s 488us/step - loss: 0.6614 - acc: 0.5000 - val_loss: 0.7180 - val_acc: 0.5000
Epoch 191/200
16/16 [==============================] - 

In [32]:
# 都预测为0.5的情况下，loss = 0.69314
pred = model.predict(train_features, batch_size=None, verbose=0, steps=None)
pred = pred.clip(min=0.005, max=0.995)

print(train_labels[:10])
print(pred.flatten()[:10])

[1 0 1 0 1 1 0 1 0 0]
[ 0.52888465  0.45712334  0.53788143  0.4516083   0.43594187  0.47686675
  0.44076961  0.54010314  0.47811815  0.45089206]
